In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Leipzig/Divergence_Indicator_2_0/Data/Data_Final'

Mounted at /content/drive


In [ ]:
import statsmodels.api as sm
from scipy.signal import convolve

def vech(x):
    """
    Transform a square matrix into a vector containing the lower triangular part.
    Returns a vector of length k*(k+1)/2 containing elements from the lower triangular part of x.
    """
    #return x[np.tril_indices(x.shape[0])]
    rows, cols = x.shape
    return np.hstack([x[i:, i] for i in range(cols)])

def ivech(data):
    """
    Transform a vector into a lower triangular matrix.
    Returns a k by k lower triangular matrix.
    """
    t = len(data)
    # Calculate the size of the output matrix
    sizeout = int((-1 + np.sqrt(1 + 8 * t)) / 2)
    transformeddata = np.zeros((sizeout, sizeout))

    index = 0
    for i in range(sizeout):
        for j in range(i, sizeout):
            transformeddata[j, i] = data[index]
            index += 1

    return transformeddata

def myfun(x, FCyc_s, FCyc_us):
    """
    Objective function for optimization, calculating the sum of squared differences.
    Returns the sum of squared differences between the modeled and actual data.
    """
    return np.sum((FCyc_s * x[0] + x[1] - FCyc_us) ** 2)

def fn_forecast_ar(temp, p_min=1, p_max=8): # Tested and works
    """
    Forecast future values of a time series using an autoregressive (AR) model.
    Returns:
    - dY_w_fcast (numpy.ndarray): The original series with the forecasted values appended.
    - length_fcast (int): The number of forecasted values.
    """
    # Determine the number of forecasts to make
    T = len(temp)
    length_fcast = min(int(2 * T / 3), 40)

    # Fit the AR model within the specified lag range
    model = sm.tsa.arima.ARIMA(temp, order=(p_max, 0, 0), trend='n')
    results = model.fit()

    # Forecast future values
    forecast = results.forecast(steps=length_fcast)

    # Combine the original data with the forecasted values
    dY_w_fcast = np.concatenate([temp, forecast])

    return dY_w_fcast, length_fcast

def undrift(x, T, nvars):
    """Removes drift from the data."""
    xun = np.zeros_like(x)
    dd = np.arange(T)[:, None]
    for ivar in range(nvars):
        drift = (x[-1, ivar] - x[0, ivar]) / (T - 1)
        xun[:, ivar] = x[:, ivar] - dd * drift
    return xun

def fn_bpass_all(X, pl, pu, root=0, drift=0, ifilt=0, nfix=-1, thet=1): # Tested and works
    # Calculate the angular frequencies for pl and pu
    b = 2 * np.pi / pl
    a = 2 * np.pi / pu

    T = len(X)

    # Prepare the theta array and compute the convolution to create the g array
    if not isinstance(thet, np.ndarray):
        thet = np.array([thet])
    nq = len(thet) - 1
    thet_padded = np.pad(thet, (0, nq), 'constant')
    g = convolve(thet, thet_padded[::-1], 'full')
    cc = g[nq:2*nq+1]

    # Construct the ideal response matrix B
    j = np.arange(1, 2 * T + 1)
    B = np.hstack(((b - a) / np.pi, (np.sin(j * b) - np.sin(j * a)) / (j * np.pi)))

    # Compute R using the closed-form integral solution
    R = np.zeros(T)
    R0 = B[0] * cc[0] + 2 * np.dot(B[1:nq+1], cc[1:nq+1])
    R[0] = np.pi * R0
    for i in range(1, T):
        R[i] = R[i - 1] - 2 * np.pi * B[i] * cc[0]

    # Initialize the AA matrix
    AA = np.zeros((T, T))

    if ifilt == 0:  # asymmetric filter
        for i in range(T):
            AA[i, i:T] = B[:T-i]
            if root == 1:
                AA[i, T-1] = R[T-i] / (2 * np.pi)

        # Use symmetry to construct the bottom half of the AA matrix
        AA[0, 0] = AA[-1, -1]
        AA = AA + np.triu(AA, 1).T

    if drift > 0:
        X = undrift(X, T, nvars)

    # Apply the filter matrix AA to the data matrix X
    fX = np.dot(AA, X)

    return fX

In [ ]:
def fn_grid(q_low, q_high, resolution):
    """
    Produces the grid along which the spectral density is estimated.
    Step defines the x-axis for graphs.
    """
    omega_step = 2048
    pi = np.pi

    if q_low == 0 and q_high != 0:
        grid = np.arange(2 * pi / q_high, pi, 2 * pi / omega_step)
    elif q_low != 0 and q_high == 0:
        grid = np.arange(0, (2 * pi) / q_low, 2 * pi / omega_step)
    elif q_low == 0 and q_high == 0:
        grid = np.arange(0, pi, 2 * pi / omega_step)
    else:
        grid = np.arange(2 * pi / q_high, (2 * pi) / q_low, 2 * pi / omega_step)

    step = (max(grid) - min(grid)) / resolution  # define steps for frequency display in graphs
    return grid, step

In [ ]:
from scipy.signal import windows, correlate
from numpy.fft import fft, ifft

def parzenwin(window_len):
    """Generate a Parzen window for given window length"""
    return windows.parzen(window_len)

def pw_cohesion(dY, grid, factor, idx):
    dy = dY[:, np.array(idx, dtype=bool)]
    M = dy.shape[1]

    f_hat_save = np.zeros(len(grid))
    f_hat_save_separate = []

    kk = 0
    for ii in range(M - 1):
        for jj in range(ii + 1, M):
            kk += 1
            y = dy[:, ii]
            x = dy[:, jj]

            # Clean missing or invalid data points
            valid = ~(np.isnan(x) | np.isnan(y))
            x = x[valid]
            y = y[valid]

            T = len(y)
            gammahat = correlate(y, x, mode='full') / T
            gammahat /= (np.std(y) * np.std(x))
            t_cov = len(gammahat)
            Mwind = round(np.sqrt(len(y)) * factor)
            w = parzenwin(2 * Mwind + 1)
            ti = t_cov // 2
            tt = np.arange(ti + Mwind, ti - Mwind - 1, -1)
            cov_trunc = gammahat[tt] * w

            # Fourier Transform
            f_hat = (2 / (2 * np.pi)) * np.abs(np.dot(cov_trunc, np.exp(-1j * np.arange(-Mwind, Mwind + 1)[:, None] * grid)))

            f_hat_save += f_hat
            f_hat_save_separate.append(f_hat * (np.std(y) * np.std(x)))

    f_hat_save /= kk

    return f_hat_save, np.array(f_hat_save_separate)


def extrema(in_data):
    dsize = len(in_data)
    flag = 1
    spmax = [[1, in_data[0]]]
    spmin = [[1, in_data[0]]]

    # Find local maxima
    for jj in range(1, dsize-1):
        if in_data[jj-1] <= in_data[jj] >= in_data[jj+1]:
            spmax.append([jj + 1, in_data[jj]])
        if in_data[jj-1] >= in_data[jj] <= in_data[jj+1]:
            spmin.append([jj + 1, in_data[jj]])

    # Handle end points for maxima and minima
    spmax.append([dsize, in_data[-1]])
    spmin.append([dsize, in_data[-1]])

    # Handle spline end effects if there are enough points
    if len(spmax) >= 4:
        adjust_endpoint(spmax)
    if len(spmin) >= 4:
        adjust_endpoint(spmin, mode='min')

    return np.array(spmax), np.array(spmin), flag

def adjust_endpoint(sp, mode='max'):
    # Adjust end point based on slope
    slope1 = (sp[1][1] - sp[2][1]) / (sp[1][0] - sp[2][0])
    tmp1 = slope1 * (sp[0][0] - sp[1][0]) + sp[1][1]
    if mode == 'max' and tmp1 > sp[0][1]:
        sp[0][1] = tmp1
    elif mode == 'min' and tmp1 < sp[0][1]:
        sp[0][1] = tmp1

    slope2 = (sp[-2][1] - sp[-3][1]) / (sp[-2][0] - sp[-3][0])
    tmp2 = slope2 * (sp[-1][0] - sp[-2][0]) + sp[-2][1]
    if mode == 'max' and tmp2 > sp[-1][1]:
        sp[-1][1] = tmp2
    elif mode == 'min' and tmp2 < sp[-1][1]:
        sp[-1][1] = tmp2



In [ ]:
from scipy.integrate import trapezoid

def fn_integral_min_dist(coh, peak, grid, pct):
    """Computes the percentage of the area below the spectral density with minimal distance"""

    total_area = trapezoid(coh)

    output_summary = np.zeros((len(grid)-1, 6))
    for ii in range(len(grid)-1):
        jj = 1
        share = trapezoid(coh[ii:ii+jj+1])
        while share / total_area <= pct and ii + jj != len(grid) - 1:
            jj += 1
            share = trapezoid(coh[ii:ii+jj+1])

        output_summary[ii, :] = [ii, jj + ii, (grid[ii])**(-1) * np.pi/2, (grid[jj + ii])**(-1) * np.pi/2, jj, share / total_area]

    # Applying filters to find the valid ranges according to the specified conditions
    temp = output_summary[output_summary[:, -1] > pct]  # filter via the pct rule
    temp = temp[temp[:, 2] >= peak]  # ensure peak is included
    temp = temp[temp[:, 3] <= peak]  # ensure peak is included

    # Select the entry with the minimum distance in grid points
    min_distance = temp[temp[:, 4] == min(temp[:, 4])]

    if len(min_distance) > 0:
        min_distance = min_distance[-1, :]  # taking the one with the highest frequency (or last in array if tied)
    else:
        min_distance = None  # handle case where no valid segment is found

    return min_distance

In [ ]:
from scipy.stats import norm
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.optimize import minimize
from numpy import nanmean, nanstd

def fn_cycle_extract_ecdf_aggr(dY, idx, min_distance, corr_rol, sgn_res):
    d = sum(idx)
    idx = np.array(idx, dtype=bool)
    y = dY[:, idx]
    temp = np.full_like(y, np.nan)
    fdY = np.full_like(y, np.nan)
    temp_indicator_smooth = np.full_like(y, np.nan)
    FCyc_sav_unsmooth = np.full((y.shape[0], 1), np.nan)
    FCycle = np.full((y.shape[0], 1), np.nan)
    weights = np.full_like(y, np.nan)
    C_out = np.full((y.shape[0], int(d*(d+1)/2)), np.nan)

    for ii in range(d):
        valid_indices = ~np.isnan(y[:, ii])
        temp2 = y[valid_indices, ii]
        temp2_fcast, length_fcast = fn_forecast_ar(temp2)

        temp10 = fn_bpass_all(temp2_fcast - np.mean(temp2_fcast), min_distance[3]*4, min_distance[2]*4,0,0,0)
        temp10 = temp10[:len(temp10) - length_fcast] + np.mean(temp2)

        # ECDF mapping
        ecdf_function = ECDF(temp2)
        xx = ecdf_function.x[1:]
        yy = ecdf_function.y[1:]

        # Find unique values and indices
        unique_values, li = np.unique(temp2, return_inverse=True)
        idx_nan = np.isnan(temp[:, ii])

        temp[idx_nan, ii] = yy[li]
        # Assign 'temp10' to 'fdY' where 'idx_nan' is True
        fdY[idx_nan, ii] = temp10

        # Optimization
        x0 = [1, 0]
        res = minimize(lambda x: myfun(x, temp10, yy[li]), x0, method='L-BFGS-B',
                       options={'maxfun': 50000, 'maxiter': 50000, 'eps': 1.0e-5, 'ftol': 1.0e-5, 'disp': False})
        x = res.x
        temp_adjust = temp10 * x[0] + x[1]

        temp_adjust[temp_adjust > 1] = 1  # Cut off max and min
        temp_adjust[temp_adjust < 0] = 0

        nan_indices = np.where(idx_nan)[0]
        col_index = [ii]
        temp_indicator_smooth[nan_indices, col_index] = temp_adjust

    dY_ecdf = temp
    idx_cyc_sav = np.all(~np.isnan(temp), axis=1)
    temp = temp[idx_cyc_sav, :]
    C = np.ones((d, d, len(temp)))
    C_vech = np.ones((len(temp), d * (d + 1) // 2))
    FCyc_sav = np.full((len(temp), 1), np.nan)
    weights_sav = np.full((len(temp), d), np.nan)

    if corr_rol == 1:
        lam = 0.89  # Smoothing parameter
        sigma_ij = np.zeros((len(temp) + 1, d * (d + 1) // 2))
        # Initialize covariance using the first 8 rows and vectorize the lower triangular part
        initial_cov = np.nan_to_num(np.cov(temp[:8, :], rowvar=False))
        sigma_ij[0, :] = vech(initial_cov)

        for ii in range(len(temp)):
            ttt = 0
            sgn_sigma = np.zeros((1, int(d * (d + 1) / 2)))
            for jj in range(d):
              for kk in range(jj, d):
                  ttt += 1
                  sigma_ij[ii + 1, ttt - 1] = lam * sigma_ij[ii, ttt - 1] + (1 - lam) * (temp[ii, jj] - 0.5) * (temp[ii, kk] - 0.5)
                  if sigma_ij[ii + 1, ttt - 1] > 0:
                    sgn_sigma[0, ttt - 1] = sigma_ij[ii + 1, ttt - 1]

            if sgn_res == 1:
                temp_vech = ivech(sgn_sigma.flatten())  # Convert the row matrix to a vector
            else:
                temp_vech = ivech(sigma_ij[ii + 1, :])

        # Compute covariance and correlation matrices
            temp_cov = temp_vech + temp_vech.T - np.diag(np.diag(temp_vech)) # Covariance
            temp_sqrt_diag = np.linalg.inv(np.sqrt(np.diag(np.diag(temp_vech)))) # Correlation
            C[:, :, ii - 1] = temp_sqrt_diag @ temp_cov @ temp_sqrt_diag
            C_vech[ii - 1, :] = vech(np.squeeze(C[:, :, ii - 1]))
            FCyc_sav[ii - 1, 0] = (np.sum(np.squeeze(C[:, :, ii - 1]) * temp[ii])) / np.sum(np.sum(np.squeeze(C[:, :, ii - 1]))) # Correlation weighted indices
            weights_sav[ii - 1] = np.sum(np.squeeze(C[:, :, ii - 1]), axis=0) / np.sum(np.sum(np.squeeze(C[:, :, ii - 1])))
        weights[idx_cyc_sav, :] = weights_sav

    else:
        FCyc_sav = np.mean(temp, axis=1)
        #return FCyc_sav, weights

    # Using ECDF on the filtered data
    temp = FCyc_sav
    ecdf_function = ECDF(temp.flatten())
    xx = ecdf_function.x[1:]  # The unique sorted values
    ff = ecdf_function.y[1:]  # The ECDF values at those points

    # Mapping unique values from 'temp' to corresponding ECDF values
    unique_temp, li = np.unique(temp, return_inverse=True)
    FCyc_sav_unsmooth[idx_cyc_sav, 0] = ff[li]

    temp_fcast, length_fcast = fn_forecast_ar(ff[li])

    # Band-pass filtering the forecasted data
    temp_fcast_mean_adjusted = temp_fcast - np.mean(temp_fcast)
    FCyc_sav_smooth = fn_bpass_all(temp_fcast_mean_adjusted, min_distance[3] * 4, min_distance[2] * 4, 0, 0, 0)

    # Calculating the mean of the adjusted ECDF values
    xx_mu = np.mean(ff[li])

    # Adjusting the smooth data by adding the mean back to the filtered data, truncated by the forecast length
    FCyc_sav_smooth = FCyc_sav_smooth[:-length_fcast] + xx_mu

    # Optimization
    x0 = [1, 0]
    res = minimize(lambda x: myfun(x, FCyc_sav_smooth, ff[li]), x0, method='L-BFGS-B',
                   options={'maxfun': 50000, 'maxiter': 50000, 'eps': 1.0e-5, 'ftol': 1.0e-5, 'disp': False})
    x = res.x
    temp_adjust = FCyc_sav_smooth * x[0] + x[1]

    temp_adjust = np.clip(temp_adjust, 0, 1)

    # Assigning the adjusted values to FCycle based on index
    FCycle[idx_cyc_sav] = temp_adjust.reshape(-1, 1)

    # Mapping C_out values based on the index
    C_out[idx_cyc_sav, :] = C_vech[idx_cyc_sav, :]

    # Constructing series_graph from FCycle and temp_indicator_smooth
    series_graph = np.hstack([FCycle, temp_indicator_smooth])
    return [series_graph,FCycle,dY_ecdf,C_out,FCyc_sav_unsmooth,weights]


In [ ]:
path

'/content/drive/MyDrive/Leipzig/Divergence_Indicator_2_0/Data/Data_Final'

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

cpi_data =pd.read_excel(path+'/Data_Overview_29082024.xlsx',sheet_name='CPI')
cpi_data_country=cpi_data['Country']
for i in range(len(cpi_data_country)):
    try:
        df_ger_bus = pd.read_excel(path+'/business_cycle_data_final_ERMcut.xlsx',index_col= 'Unnamed: 0',
                          sheet_name=cpi_data_country[i])
        df_ger_fin = pd.read_excel(path+'/financial_cycle_data_final_ERMcut.xlsx',index_col= 'Unnamed: 0',
                          sheet_name=cpi_data_country[i])
        df_ger_bus
        cpi = pd.read_excel(path+'/CPI.xlsx',index_col='Unnamed: 0')
        cpi =cpi[cpi_data_country[i]]

        Date_start=cpi_data.loc[cpi_data['Country'] == cpi_data_country[i],'Dates (ERM)'].iloc[0]
        df_ger_bus = df_ger_bus[['private consumption', 'unemployment', 'GDP',
        'gross fixed capital formation']] # Rearrange
        df_ger_fin = df_ger_fin[['credit', 'house price index', 'share price (equity)',
        'government bond yield']]
        common_index = df_ger_bus.index.intersection(df_ger_fin.index)

        # Filter the dataframes to include only the common index
        df_ger_bus = df_ger_bus.loc[common_index]
        df_ger_fin = df_ger_fin.loc[common_index]
        cpi = cpi.loc[common_index]
        dates = pd.to_datetime(df_ger_bus.index)
        df_ger_bus_year = dates.year
        df_ger_fin_year = dates.year
        df_ger_bus = df_ger_bus[df_ger_bus_year >= Date_start]
        df_ger_fin = df_ger_fin[df_ger_fin_year >= Date_start]


        df_ger_bus.dropna(axis=0, how='any', inplace=True)
        df_ger_fin.dropna(axis=0, how='any', inplace=True)

        common_index = df_ger_bus.index.intersection(df_ger_fin.index)

        # Filter the dataframes to include only the common index
        df_ger_bus = df_ger_bus.loc[common_index]
        df_ger_fin = df_ger_fin.loc[common_index]
        cpi = cpi.loc[common_index]
        # print(cpi)
        # print(len(cpi))
        df_ger_bus_1 = df_ger_bus[['private consumption', 'unemployment', 'GDP']].div(cpi, axis=0)*100
        df_ger_fin_1= df_ger_fin[['credit', 'house price index', 'share price (equity)']].div(cpi, axis=0)*100
        df_ger_bus_2 = df_ger_bus[['gross fixed capital formation']]
        df_ger_fin_2= df_ger_fin[['government bond yield']].div((1+df_ger_fin[['government bond yield']]),axis=0)
        df_ger_fin_2= df_ger_fin_2.div(cpi,axis=0)*100
        df_ger_bus = df_ger_bus_1.join(df_ger_bus_2)
        df_ger_fin = df_ger_fin_1.join(df_ger_fin_2)
        #print(df_ger_fin)

        # Calculate percentage change for df_ger_bus
        df_ger_bus = df_ger_bus.pct_change()

        # Calculate percentage change for df_ger_fin

        if cpi_data_country[i] == 'Malta':
            # Exclude 'share price (equity)' for Malta when calculating pct_change
            df_ger_fin_except_equity = df_ger_fin.drop(columns=['share price (equity)']).pct_change()
            df_ger_fin['share price (equity)'] = df_ger_fin['share price (equity)']  # Keep 'share price (equity)' as is
            df_ger_fin.update(df_ger_fin_except_equity)  # Update df_ger_fin with pct_change for other columns
        else:
            # Apply pct_change to the entire dataframe if not Malta
            df_ger_fin = df_ger_fin.pct_change()

        #df_ger_fin = df_ger_fin.pct_change()

        df_ger_bus_ = df_ger_bus.iloc[1:-1,:]
        df_ger_fin_ = df_ger_fin.iloc[1:-1,:]
        df_ger_bus_.replace([np.inf, -np.inf], 0, inplace=True)
        df_ger_fin_.replace([np.inf, -np.inf], 0, inplace=True)
        print(df_ger_bus_.isna().sum().sum())
        print(df_ger_fin_.isna().sum().sum())

        dates = pd.to_datetime(df_ger_bus_.index)
        df_ger_bus_= df_ger_bus_.reset_index(drop=True)
        df_ger_fin_= df_ger_fin_.reset_index(drop=True)
        df_ger_bus_np = df_ger_bus_.to_numpy()
        df_ger_fin_np = df_ger_fin_.to_numpy()

        centered_data_bus = df_ger_bus_np - df_ger_bus_np.mean()
        centered_data_fin = df_ger_fin_np - df_ger_fin_np.mean()

        dY_bus = df_ger_bus_np
        dY_fin = df_ger_fin_np

        co = [20]            # number of countries
        n_co = len(co)
        T = dY_bus.shape[0]


        fdY_bus = fn_bpass_all(centered_data_bus,2,200,0,0,0)
        fdY_fin = fn_bpass_all(centered_data_fin,2,200,0,0,0)

        q_low = 5             # lower bound for spectral density, put 0 for lowest (10=benchmark)
        q_high = 200          # upper bound for spectral density, put 0 for highest (0 = benchmark)
        factor = 8            # precision of spectral density estimates
        N = 1                 # number of peaks to select
        pct = 0.67            # percentage rule for the area of power cohesion to select
        corr_rol = 1          # composite Cycle: if 1 - rolling correlations; 0 - linear average
        sgn_res = 1           # sgn restrictions CISS: emphasises positively related movements
        idx_fc = [1, 1, 1, 1]
        idx_bc = [1, 1, 1, 1]

        grid, step = fn_grid(q_low, q_high, 16)
        # FC broad
        FCycle = np.zeros((T, n_co))
        FCycle_rt = np.zeros((T, n_co))
        FCycle_unsmooth = np.zeros((T, n_co))
        FCycle_unsmooth_rt = np.zeros((T, n_co))
        series_graph_fc = np.zeros((T, n_co * 5))
        time_varying_weights =  np.zeros((T, n_co * 4))
        # FC narrow
        FCycle_n = np.zeros((T, n_co))
        FCycle_n_rt = np.zeros((T, n_co))
        series_graph_fc_n = np.zeros((T, n_co * 3))
        FCycle_unsmooth_n = np.zeros((T, n_co))
        FCycle_unsmooth_n_rt = np.zeros((T, n_co))
        # Business Cycle
        BCycle = np.zeros((T, n_co))
        BCycle_rt = np.zeros((T, n_co))
        series_graph_bc = np.zeros((T, n_co * 5))
        BCycle_unsmooth = np.zeros((T, n_co))
        BCycle_unsmooth_rt = np.zeros((T, n_co))

        fdY_fin = np.array(fdY_fin)
        print(fdY_fin.shape)
        fdY_bus = np.array(fdY_bus)
        dY_fin = np.array(dY_fin)
        dY_bus = np.array(dY_bus)

        # Initialize arrays to store peaks
        peak_fc = np.zeros((1, 1))
        peak_fc_n = np.zeros((1, 1))
        peak_bc = np.zeros((1, 1))

        # Loop across countries (here, only the US)
        for jip in range(len(co)):
            dY_f = fdY_fin  # Data for financial cycles
            dY_b = fdY_bus  # Data for business cycles
            dY_f_rt = dY_fin  # Real-time data for financial cycles
            dY_b_rt = dY_bus  # Real-time data for business cycles

            # Power cohesion calculations
            pw_coh_fc, f_hats_fc = pw_cohesion(dY_f, grid, factor,idx_fc )
            pw_coh_fc_n, f_hats_fc_n = pw_cohesion(dY_f, grid, factor, [1, 1, 0, 0])
            pw_coh_bc, f_hats_bc = pw_cohesion(dY_b, grid, factor, idx_bc)

            # Store peak frequencies
            peak_fc = np.zeros((1, N))
            peak_fc_n = np.zeros((1, N))
            peak_bc = np.zeros((1, N))


        temp1, _, _ = extrema(pw_coh_fc)
        temp15, _, _ = extrema(pw_coh_fc_n)
        temp2, _, _ = extrema(pw_coh_bc)

        temp1_indices = temp1[:, 0].astype(int)[temp1[:, 0] < len(pw_coh_fc)]
        temp15_indices = temp15[:, 0].astype(int)[temp15[:, 0] < len(pw_coh_fc_n)]
        temp2_indices = temp2[:, 0].astype(int)[temp2[:, 0] < len(pw_coh_bc)]

        # Finding the highest N maxima
        # For financial cycles (broad)
        sort_index = np.argsort(-pw_coh_fc[temp1_indices])  # Sorting in descending order
        max_index = sort_index[:N]
        peak_fc = (np.pi / 2) / grid[temp1_indices[max_index]]

        # For financial cycles (narrow)
        sort_index = np.argsort(-pw_coh_fc_n[temp15_indices])
        max_index = sort_index[:N]
        peak_fc_n = (np.pi / 2) / grid[temp15_indices[max_index]]

        # For business cycles
        sort_index = np.argsort(-pw_coh_bc[temp2_indices])
        max_index = sort_index[:N]
        peak_bc = (np.pi / 2) / grid[temp2_indices[max_index]]

        # Derive area and distance interval
        min_distance_fc = fn_integral_min_dist(pw_coh_fc, peak_fc[0], grid, pct)
        min_distance_fc_n = fn_integral_min_dist(pw_coh_fc_n, peak_fc_n[0], grid, pct)
        min_distance_bc = fn_integral_min_dist(pw_coh_bc, peak_bc[0], grid, pct)


        # Store freq windows and peaks
        table_freqband_fcycle = np.zeros((len(co), 3))  # Update dimensions as needed
        table_freqband_fcycle_n = np.zeros((len(co), 3))
        table_freqband_bcycle = np.zeros((len(co), 3))

        table_freqband_fcycle[jip] = [min_distance_fc[2], peak_fc[0], min_distance_fc[3]]
        table_freqband_fcycle_n[jip] = [min_distance_fc_n[2], peak_fc_n[0], min_distance_fc_n[3]]
        table_freqband_bcycle[jip] = [min_distance_bc[2], peak_bc[0], min_distance_bc[3]]

        # Financial Cycle: Broad
        jip=1
        [series_graph_fc[:, 5*jip-5:5*jip],FCycle[:],_,_,FCycle_unsmooth[:],time_varying_weights[:,(jip-1)*4:(jip-1)*4+4]] = fn_cycle_extract_ecdf_aggr(dY_f, idx_fc, min_distance_fc, corr_rol, sgn_res)
        # Financial Cycle: Narrow
        idx_fcn=[1, 1, 0, 0]
        [series_graph_fc_n[:, 3*jip-3:3*jip],FCycle_n[:],_, _,FCycle_unsmooth_n[:],_] = fn_cycle_extract_ecdf_aggr(dY_f, idx_fcn, min_distance_fc_n, 0, sgn_res)

        # Business Cycle
        [series_graph_bc[:, 5*jip-5:5*jip],BCycle[:],_,_,BCycle_unsmooth[:],_] = fn_cycle_extract_ecdf_aggr(dY_b, idx_bc, min_distance_bc, corr_rol, sgn_res)

        # plt.figure(figsize=(10, 6))
        plt.subplots(figsize=(30,6))
        plt.suptitle(cpi_data_country[i],fontsize=20)
        plt.subplot(1, 3, 1)
        sns.lineplot(x=dates, y=FCycle_unsmooth[:].flatten(), label='FCycle Unsmooth')
        sns.lineplot(x=dates, y=FCycle[:].flatten(), label='FCycle')
        plt.axhline(0.5, color='gray', linestyle='--', label='Zero Line')
        plt.xlabel('Year')
        plt.ylabel('Deviation from historical median growth')
        # plt.show()

        plt.subplot(1, 3, 2)
        #plt.figure(figsize=(8, 4))
        sns.lineplot(x=dates, y=FCycle_unsmooth_n[:].flatten(), label='FCycle Unsmooth_n')
        sns.lineplot(x=dates, y=FCycle_n[:].flatten(), label='FCycle_n')
        plt.axhline(0.5, color='gray', linestyle='--', label='Zero Line')
        plt.xlabel('Year')
        plt.ylabel('Deviation from historical median growth')
        # plt.show()

        plt.subplot(1, 3, 3)
        #plt.figure(figsize=(8, 4))
        sns.lineplot(x=dates, y=BCycle_unsmooth[:].flatten(), label='BCycle Unsmooth')
        sns.lineplot(x=dates, y=BCycle[:].flatten(), label='BCycle')
        plt.axhline(0.5, color='gray', linestyle='--', label='Zero Line')
        plt.xlabel('Year')
        plt.ylabel('Deviation from historical median growth')
        plt.show()

        cycle_data = pd.DataFrame({
            'FCycle_unsmooth': FCycle_unsmooth[:].flatten(),
            'FCycle': FCycle[:].flatten(),
            'FCycle_unsmooth_n': FCycle_unsmooth_n[:].flatten(),
            'FCycle_n': FCycle_n[:].flatten(),
            'BCycle_unsmooth': BCycle_unsmooth[:].flatten(),
            'BCycle': BCycle[:].flatten(),
        }, index=dates)

        # Create the Excel file path
        #excel_file_path = os.path.join(path, 'Country_Cycle_Data', f'{cpi_data_country[i]}_Cycle_Data.xlsx')

        # Write the DataFrame to an Excel file with the index as dates
        with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a' if i > 0 else 'w') as writer:
            cycle_data.to_excel(writer, sheet_name=cpi_data_country[i], index=True)

    except Exception as e:
        print(f"Error processing {cpi_data_country[i]}: {e}")
        continue


Output hidden; open in https://colab.research.google.com to view.